This is the code that I've used to scrape the rps-episodes (Python wrapped in bash).  For CREDITS see the top of the Python, this isn't my code.

There are two known bugs:
1. If you set MIN_FINAL_RATING too high then it doens't use all of the 3600 API calls
2. It can fail when data is sparse

This notebook from: https://www.kaggle.com/tonyrobinson/rps-episode-scraper

In [3]:
import pandas as pd
import numpy as np
import os
import requests
import json
import datetime
import time
import itertools

MIN_FINAL_RATING = 1200 # top submission in a match must have reached this score
global num_api_calls_today
num_api_calls_today = 0
DIR = 'episode_scraping/episodes'
MAXAPICALLS = 3000 # this is the maximum per day permitted by the API, actual max is 3600
BUFFER = 1

base_url = "https://www.kaggle.com/requests/EpisodeService/"
get_url = base_url + "GetEpisodeReplay"
list_url = base_url + "ListEpisodes"

# inital team list
# 5981881 is nagiss
r = requests.post(list_url, json = {"teamId": 5981881}) # arbitrary ID, change to leading ID during challenge
rj = r.json()
teams_df = pd.DataFrame(rj['result']['teams'])

teams_df.sort_values('publicLeaderboardRank', inplace = True)
# Selecting the top n teams
teams_df = teams_df.iloc[:20,:]
teams_df

,id,teamName,competitionId,teamLeaderId,isBenchmark,createdAfterDeadline,medal,dateMedalAwarded,submissionCount,lastSubmissionDate,publicLeaderboardSubmissionId,publicLeaderboardScore,publicLeaderboardRank,privateLeaderboardSubmissionId,privateLeaderboardScore,privateLeaderboardRank,competition,teamMembers,inboundMergeRequests,outboundMergeRequests
10,5981881,nagiss,24539,2144394,False,False,0,None,135,"{'seconds': 1609727177, 'nanos': 606666700}",18775164,None,1,None,None,None,None,"[{'id': 2144394, 'displayName': 'nagiss', 'ema...",[],[]
68,5987921,BX,24539,1963268,False,False,0,None,59,"{'seconds': 1609733456, 'nanos': 280000000}",18927556,None,2,None,None,None,None,"[{'id': 1963268, 'displayName': 'Bill Xu', 'em...",[],[]
56,5986780,1 Million Monkeys,24539,3652230,False,False,0,None,140,"{'seconds': 1609804585, 'nanos': 233333300}",18942747,None,3,None,None,None,None,"[{'id': 2477041, 'displayName': 'px wang j', '...",[],[]
89,5995495,Kun Hao Yeh,24539,1264580,False,False,0,None,41,"{'seconds': 1609728577, 'nanos': 623333300}",18909757,None,4,None,None,None,None,"[{'id': 1264580, 'displayName': 'Kun Hao Yeh',...",[],[]
34,5984536,Ehsan,24539,600346,False,False,0,None,121,"{'seconds': 1609767869, 'nanos': 980000000}",18949158,None,5,None,None,None,None,"[{'id': 600346, 'displayName': 'Ehsan', 'email...",[],[]
112,6006860,Joan_BA,24539,5486745,False,False,0,None,84,"{'seconds': 1609833964, 'nanos': 183333300}",18898705,None,6,None,None,None,None,"[{'id': 5486745, 'displayName': 'Joan_BA', 'em...",[],[]
38,5985236,Xrc,24539,281494,False,False,0,None,121,"{'seconds': 1609833676, 'nanos': 683333300}",18751161,None,7,None,None,None,None,"[{'id': 281494, 'displayName': 'Costas Voglis'...",[],[]
141,6037087,Your mistletoe is no match for my TOW Missile!,24539,5015327,False,False,0,None,139,"{'seconds': 1609796483, 'nanos': 496666700}",18901796,None,8,None,None,None,None,"[{'id': 131576, 'displayName': 'Taffey Lewis',...",[],[]
99,5998274,Michael Maguire,24539,199918,False,False,0,None,102,"{'seconds': 1609804006, 'nanos': 126666700}",18905028,None,9,None,None,None,None,"[{'id': 199918, 'displayName': 'Michael Maguir...",[],[]
109,6004390,woody285,24539,6374505,False,False,0,None,16,"{'seconds': 1609787374, 'nanos': 560000000}",18923980,None,10,None,None,None,None,"[{'id': 6374505, 'displayName': 'woody285', 'e...",[],[]


In [13]:
def getTeamEpisodes(team_id,teams_df):

    r = requests.post(list_url, json = {"teamId":  int(team_id)})
    rj = r.json()

    # update teams list
    # global teams_df
    teams_df_new = pd.DataFrame(rj['result']['teams'])

    if len(teams_df.columns) == len(teams_df_new.columns) and (teams_df.columns == teams_df_new.columns).all():
        teams_df = pd.concat( (teams_df, teams_df_new.loc[[c for c in teams_df_new.index if c not in teams_df.index]] ) )
        teams_df.sort_values('publicLeaderboardRank', inplace = True)
    else:
        print('teams dataframe did not match')

    # make df
    team_episodes = pd.DataFrame(rj['result']['episodes'])
    team_episodes['avg_score'] = -1;

    agents = []
    for i in range(len(team_episodes)):
        agents.append(team_episodes['agents'].loc[i])
    not_none_agents = []
    for i in range(len(agents)):
        l = [a['updatedScore'] for a in agents[i] if a['updatedScore'] is not None]
        if l:
            not_none_agents.append(agents[i])
            
    for i in range(len(not_none_agents)):
        agents = not_none_agents[i]
        agent_scores = [a['updatedScore'] for a in agents if a['updatedScore'] is not None]
        team_episodes.loc[i, 'submissionId'] = max([a['submissionId'] for a in agents])
        team_episodes.loc[i, 'updatedScore'] = max([a['updatedScore'] for a in agents]) 
    
    team_episodes['final_score'] = team_episodes['updatedScore']
    team_episodes.sort_values('final_score', ascending = False, inplace=True)
    
    return rj, team_episodes


def saveEpisode(epid, rj):
    # request
    re = requests.post(get_url, json = {"EpisodeId": int(epid)})

    # save replay
    with open(DIR + '/{}.json'.format(epid), 'w') as f:
        f.write(re.json()['result']['replay'])


pulled_teams = {}
# populate pulled_episodes
## pulled_episodes = []
pulled_episodes = [ os.path.splitext(file)[0] for file in itertools.chain(os.listdir(DIR), os.listdir('episode_scraping/invalid')) ]

start_time = datetime.datetime.now()
r = BUFFER;

while num_api_calls_today < MAXAPICALLS:
    # pull team
    top_teams = [i for i in teams_df.id if i not in pulled_teams]
    if len(top_teams) > 0:
        team_id = top_teams[0]
    else:
        break;
        
    # get team data
    team_json, team_df = getTeamEpisodes(team_id,teams_df); r+=1;
    
    num_api_calls_today+=1
    
    print('{} calls and getting: {}'.format(num_api_calls_today, teams_df.loc[teams_df.id == team_id].iloc[0].teamName))
    
    team_df = team_df[  (MIN_FINAL_RATING is None or (team_df.final_score > MIN_FINAL_RATING))]
    
    print('   {} in score range from {} submissions'.format(len(team_df), len(team_df.submissionId.unique() ) ) )
    
    team_df = team_df[~team_df.id.isin(pulled_episodes)]        
    print('      {} remain to be downloaded or skipped'.format(len(team_df)))
        
    # pull games
    target_team_games = max(0, MAXAPICALLS - num_api_calls_today)
     
    pulled_teams[team_id] = 0
    
    i = 0
    while i < len(team_df) and pulled_teams[team_id] < target_team_games:
        epid = team_df.id.iloc[i]
        if not (str(epid) in pulled_episodes):
            try:
                saveEpisode(epid, team_json); r+=1;
                num_api_calls_today+=1
            except:
                time.sleep(20)
                i+=1;
                continue;
                
            pulled_episodes.append(epid)
            pulled_teams[team_id] += 1

            if r > (datetime.datetime.now() - start_time).seconds:
                time.sleep( r - (datetime.datetime.now() - start_time).seconds)

        i+=1;

99 calls and getting: nagiss
   552 in score range from 215 submissions
      552 remain to be downloaded
Skipping 9899217
Skipping 9898023
Skipping 9895945
Skipping 9899769
Skipping 9899284
Skipping 9898507
Skipping 9896865
Skipping 9897284
Skipping 9895518


KeyboardInterrupt: 

In [ ]:
#!/bin/bash -e                                                                                                                                                               

cd $HOME/rockPaperScissors

./episode-scraper.py

# tidy invalid responses                                                                                                                                                     
for file in `egrep -l INVALID episode/*json` ; do
  mv $file invalid
done

rm -rf upload
mkdir upload

NEPISODE=`ls episode/*json | wc -l`

cat << EOF > upload/dataset-metadata.json                                                                                                                                    
{                                                                                                                                                                            
  "title": "Rock, Paper, Scissors: Episode archive",                                                                                                                         
  "subtitle": "$NEPISODE episodes from top leaderboard teams",                                                                                                               
  "id": "tonyrobinson/rps-episode",                                                                                                                                          
  "licenses": [{"name": "CC0-1.0"}]                                                                                                                                          
}                                                                                                                                                                            
EOF                                                                                                                                                                          

tar --create --auto-compress --file upload/rps-episode.tar.gz --directory=episode .

# kaggle datasets create --path upload --public                                                                                                                              
kaggle datasets version --path upload --message "Updated to $NEPISODE episodes"

exit 0
